**Chapter 7 - Creating Your Own Flowables**

The `Flowable` class is easy to sub-class so that you can create your
own custom Flowables whenever you need to. You can look at the Flowables
that are included with ReportLab for ideas. There is also a chapter in
the ReportLab User Guide that has a fun example of creating a custom
Flowable that looks like a hand. You might find some user contributed
Flowables on ReportLab’s website as well, although I haven’t personally
seen too many of these.

In this chapter we will look at how to create two very basic custom
Flowables that should help you see how easy it is to create your own.

**Creating a Line Flowable**

Creating a line Flowable is actually quite simple. The first thing you
need to do is sub-class ReportLab’s `Flowable` class and then tell it
to draw a line. Let’s look at how we might put this together:
```python
class MyLineFlowable(Flowable):

    def __init__(self, width, height=0):
        Flowable.__init__(self)
        self.width = width
        self.height = height

    def __repr__(self):
        return "Line(w=%s)" % self.width
    
    def draw(self):
        # draw the line
        self.canv.line(0, self.height, self.width, self.height)
```
Here we create a pretty simply class. It accepts a `width` and a
`height` as its sole arguments. Then we override the `__repr__` method to make the
output from printing our instances of this Flowable
more readable. Finally we override the `draw` method to just draw a
line on the Flowable’s `canvas` using the width and height that were
passed in.

In [1]:
# line_flowable_demo.py

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.platypus import Flowable, Paragraph, SimpleDocTemplate, Spacer

class MyLineFlowable(Flowable):

    def __init__(self, width, height=0):
        Flowable.__init__(self)
        self.width = width
        self.height = height

    def __repr__(self):
        return "Line(w=%s)" % self.width
    
    def draw(self):
        # draw the line
        self.canv.line(0, self.height, self.width, self.height)


def create_line_flowable():
    """
    Create a pdf
    """
    story = []
    doc = SimpleDocTemplate("create_line_flowable.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    spacer = Spacer(0, 0.25*inch)

    ptext = '<font size=12>%s</font>' % "Section #1"
    story.append(Paragraph(ptext, styles["Normal"]))
    story.append(spacer)

    line = MyLineFlowable(500)
    story.append(line)
    story.append(spacer)

    ptext = '<font size=12>%s</font>' % "Section #2"
    story.append(Paragraph(ptext, styles["Normal"]))
    
    doc.build(story)

if __name__ == "__main__":
    create_line_flowable()

**Creating a Bordered Textbox + Line Flowable**

During one of my jobs, I needed to create a text box with a border plus
a line that went from the top of the box off to the right and I needed
to be able to add it to my document as a Flowable. It kind of looked
like this piece of ASCII art:
```ascii
-----------------------------------------
| foobar |
----------
```
This time when we create our class, we set it up to take five arguments:
x, y, width, height and text. The x and y are for helping in positioning
the Flowable’s lines correctly. The width and height are for setting the
rectangle’s dimensions and the text is what will be displayed in the
box.
<br></br>
One of the immediate improvements you
could make to this Flowable would be to calculate the width of the
string that was passed into it and adjust the bounding rectangle
accordingly. You could accomplish that fairly easily by using the
`stringWidth` function from `reportlab.pdfbase.pdfmetrics`.

In [2]:
# boxy_line_demo.py

from reportlab.lib.pagesizes import letter
from reportlab.platypus import Flowable, SimpleDocTemplate, Spacer
from reportlab.lib.units import inch

class BoxyLine(Flowable):
    """
    Draw a box + line + text
    -----------------------------------------
    | foobar |
    ---------
    """
    def __init__(self, x=0, y=-15, width=40, height=15, text=""):
        Flowable.__init__(self)
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.text = text

    def draw(self):
        """
        Draw the shape, text, etc
        """
        self.canv.rect(self.x, self.y, self.width, self.height)
        self.canv.line(self.x, 0, 500, 0)
        self.canv.drawString(self.x+5, self.y+3, self.text)


def create_boxy_line_flowable():
    """
    Create a pdf
    """
    doc = SimpleDocTemplate("boxy_line_flowable.pdf",pagesize=letter)
    story=[]
    box = BoxyLine(text="foo")
    story.append(box)
    story.append(Spacer(0, 1*inch))
    box = BoxyLine(text="bar")
    story.append(box)
    doc.build(story)


if __name__ == "__main__":
    create_boxy_line_flowable()

**Modifying a Pre-Existing Flowable**

Sometimes all you really want to do is make an existing Flowable behave
in a way that the default version doesn’t appear to do. You can easily
subclass a pre-existing Flowable like the `Paragraph` or the `Image` and just
override its methods to make it do something different.
Just take a look at the Flowable you would like to change and see what
methods it has. Then you can make an educated guess which ones you will
need to change to make your new Flowable work the way you want it to.
The ReportLab User Guide has one example where they subclassed the
`Image` class to make it draw all images at a 90 degree angle, for
example.